# Bayesian Regression & MLE for 

**Team member**: Hung Dinh - 19774520

**Theme**: Bayesian vs Frequentist Approach on Regression - Student Performance

**Github Repo**: https://github.com/hd54/stat447c

All commits are done by me and me only.

**Introduction**:

Performance have always been a concern for a lot of students regardless of education level, whether it be high school, university, or college. Good performance can mean greater opportunities for higher education, awards, and even jobs, so students want to be successful in their courses. However, there's always a disparity in students' performance, which can be seen in grade distributions of exams, homework, etc. It's possible that students' background or how they treat the class affect their performance. This project seeks to see how different factors contribute to students' performance (in particular, final exam score or course grade letter).

**Dataset**:

Student performance may vary throughout the years due to societal changes, introduction of new technologies (such as ChatGPT), etc. This leads me to choose some of the more recent datasets as possible candidates:

https://www.kaggle.com/datasets/lainguyn123/student-performance-factors

https://www.kaggle.com/datasets/joebeachcapital/students-performance

In [ ]:
library(tidyverse)

**Approaches**:



I can start simple with a comparative study of different types of regression using frequentist and Bayesian approach. I can do a comparative study on traditional (without penalty) and regularized regression for each approach. For frequentist approach, I can start by optimizing the number of variables used for regression through removing collinearity (i.e. we can use forward selection along with VIF analysis). I would expect to see regularized model to perform at least as well on predicting data due to the extra penalty added.

For Bayesian approach, 

If given enough time, I can also study interaction effects, but I can achieve a (hopefully) reasonable multilinear regression model.

**References**:
https://haines-lab.com/post/on-the-equivalency-between-the-lasso-ridge-regression-and-specific-bayesian-priors/